In [2]:
!cd

C:\tmp\stooq


In [3]:
import os
import glob
import time
from pathlib import Path
import pandas as pd
import duckdb

In [4]:
conn = duckdb.connect("today.duckdb")

In [10]:
file_data = "20230217_dh5.txt"

## CSV Import

https://duckdb.org/docs/guides/import/csv_import

In [15]:
cur_time = time.time()
df = conn.execute(f"""
	SELECT *
	FROM read_csv_auto('{file_data}', ALL_VARCHAR=1);
""").df()
print(f"time: {(time.time() - cur_time)}")

time: 0.08532261848449707


In [16]:
df.head(5)

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,<OPENINT>
0,^AEX,5,20230217,090500,763.3,763.78,762.71,762.9,0,0
1,^AEX,5,20230217,091000,762.87,763.41,762.46,762.46,0,0
2,^AEX,5,20230217,091500,762.49,762.52,761.79,761.83,0,0
3,^AEX,5,20230217,092000,761.76,762.54,761.72,762.11,0,0
4,^AEX,5,20230217,092500,762.18,762.51,761.79,762.51,0,0


In [17]:
df.columns

Index(['<TICKER>', '<PER>', '<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>',
       '<CLOSE>', '<VOL>', '<OPENINT>'],
      dtype='object')

In [18]:
col_map = { c:c[1:-1] for c in df.columns}

In [19]:
col_map

{'<TICKER>': 'TICKER',
 '<PER>': 'PER',
 '<DATE>': 'DATE',
 '<TIME>': 'TIME',
 '<OPEN>': 'OPEN',
 '<HIGH>': 'HIGH',
 '<LOW>': 'LOW',
 '<CLOSE>': 'CLOSE',
 '<VOL>': 'VOL',
 '<OPENINT>': 'OPENINT'}

In [20]:
df.rename(columns=col_map, inplace=True)

In [21]:
df.head(5)

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT
0,^AEX,5,20230217,090500,763.3,763.78,762.71,762.9,0,0
1,^AEX,5,20230217,091000,762.87,763.41,762.46,762.46,0,0
2,^AEX,5,20230217,091500,762.49,762.52,761.79,761.83,0,0
3,^AEX,5,20230217,092000,761.76,762.54,761.72,762.11,0,0
4,^AEX,5,20230217,092500,762.18,762.51,761.79,762.51,0,0


In [22]:
conn.execute("""
CREATE OR REPLACE TABLE daily_quote AS
	SELECT
		"TICKER" as ticker,
        "PER" as per,
        "DATE" as date_,
        "TIME" as time_,
        "OPEN"::REAL as open_,
        "HIGH"::REAL as high_,
        "LOW"::REAL as low_,
        "CLOSE"::REAL as close_,
        "VOL"::LONG as vol_,
        "OPENINT"::INT as openint_
	FROM df
	WHERE
		TRY_CAST("CLOSE" AS REAL) NOTNULL
""")

In [25]:
conn.execute("DESCRIBE daily_quote").df()

,column_name,column_type,null,key,default,extra
0,ticker,VARCHAR,YES,NaN,NaN,NaN
1,per,VARCHAR,YES,NaN,NaN,NaN
2,date_,VARCHAR,YES,NaN,NaN,NaN
3,time_,VARCHAR,YES,NaN,NaN,NaN
4,open_,FLOAT,YES,NaN,NaN,NaN
5,high_,FLOAT,YES,NaN,NaN,NaN
6,low_,FLOAT,YES,NaN,NaN,NaN
7,close_,FLOAT,YES,NaN,NaN,NaN
8,vol_,BIGINT,YES,NaN,NaN,NaN
9,openint_,INTEGER,YES,NaN,NaN,NaN


In [23]:
conn.execute("""
    SELECT COUNT(*) FROM daily_quote;
""").df()

,count_star()
0,35960


In [24]:
conn.execute("""
    SELECT per,COUNT(*) FROM daily_quote group by per;
""").df()

,per,count_star()
0,5,21970
1,D,12055
2,60,1935


In [27]:
conn.execute("""
    SELECT count(*) FROM daily_quote 
    where per='D' and vol_ > 10000
    ;
""").df()

,count_star()
0,9011


In [29]:
conn.execute("""
    SELECT * FROM daily_quote 
    where per='D' and vol_ > 100000
    and ticker like '^%'
    ;
""").df()

,ticker,per,date_,time_,open_,high_,low_,close_,vol_,openint_
0,^BUX,D,20230217,000000,46040.480469,46040.480469,45485.320312,45626.921875,1333506,0
1,^DJC,D,20230217,000000,11385.370117,11453.070312,11345.849609,11443.160156,436013557,0
2,^DJI,D,20230217,000000,33677.011719,33846.640625,33517.730469,33826.691406,230868749,0
3,^IBEX,D,20230217,000000,9276.299805,9363.099609,9240.799805,9333.000000,168393650,0
4,^KOSPI,D,20230217,000000,2447.659912,2471.510010,2445.739990,2451.209961,507560000,0
5,^NDQ,D,20230217,000000,11777.500000,11803.219727,11673.209961,11787.269531,3521810510,0
6,^NDX,D,20230217,000000,12352.299805,12385.519531,12233.790039,12358.179688,744565285,0
7,^SHBS,D,20230217,000000,294.688690,296.029694,294.157196,294.531189,27984200,0
8,^SHC,D,20230217,000000,3244.732422,3262.472900,3223.258057,3224.024414,27845123700,0
9,^SPX,D,20230217,000000,4077.389893,4081.510010,4047.949951,4079.090088,1835332862,0


In [30]:
conn.execute("""
    SELECT * FROM daily_quote 
    where per='D' and vol_ > 100000
    and ticker not like '^%'
    limit 10
    ;
""").df()

,ticker,per,date_,time_,open_,high_,low_,close_,vol_,openint_
0,AAU.US,D,20230217,000000,0.2230,0.2250,0.2192,0.2250,398537,0
1,AIM.US,D,20230217,000000,0.4840,0.5150,0.4818,0.5150,263614,0
2,ASM.US,D,20230217,000000,0.7250,0.7557,0.7150,0.7300,434526,0
3,ASXC.US,D,20230217,000000,1.0100,1.0300,0.9010,0.9749,3025666,0
4,ATNM.US,D,20230217,000000,14.4500,14.4514,13.6201,13.7600,723553,0
5,AULT.US,D,20230217,000000,0.1326,0.1337,0.1210,0.1234,3581271,0
6,AUMN.US,D,20230217,000000,0.2480,0.2549,0.2421,0.2490,237685,0
7,BMTX.US,D,20230217,000000,3.8400,3.8400,3.5900,3.6100,103651,0
8,BTG.US,D,20230217,000000,3.3400,3.3400,3.2600,3.3200,8151019,0
9,BURU.US,D,20230217,000000,4.3000,4.5200,3.8000,3.8000,111396,0


In [31]:
conn.commit()

In [32]:
conn.close()